In [1]:
# !pip install -r requirements.txt

In [13]:
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
import logging
import multiprocessing
from datetime import datetime, timedelta

from src import modules as f
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
# Download, load, extract, model...
interval = '1d'
stock_set = f.download_interval_process(interval, 1)
print(f'Stock set: \n{stock_set}\n')

Stock set: 
{'VZ', 'BRK-B', 'AAPL', 'FWONK', 'UNH', 'AJG', 'ADBE', 'DIS', 'PLTR', 'DECK', 'SPGI', '^VIX', 'CL', 'DUK', 'ACN', 'AVGO', 'D', 'CEG', 'PM', 'MSI', 'CVX', 'AXP', 'IBM', 'PSX', 'LNG', 'AEP', 'MDLZ', 'SW', 'SRE', 'KMI', 'META', 'XOM', 'NFLX', 'KO', 'SLB', 'OKE', 'EXE', 'TDG', 'EOG', 'RCL', 'WMB', 'TPR', 'WSM', 'TSLA', 'SO', 'TGT', 'DKNG', 'BAC', 'AMZN', 'JPM', 'AMGN', 'GOOG', 'WFC', 'CRM', 'COP', 'PGR', 'FCNCA', 'MO', 'BSX', 'T', 'MA', 'TMUS', 'ABBV', 'GS', 'ORCL', 'V', 'CMCSA', 'WELL', 'MSFT', 'TTWO', 'GOOGL', 'VST', 'DASH', 'XEL', 'COST', 'EXC', 'NRG', 'NEE', 'PEP', 'ISRG', 'WMT', 'MRK', 'HD', 'NVDA', 'APH', 'PG', 'LLY', 'JNJ', 'CSCO', 'ABT', 'TMO', 'ATO'}



In [4]:
f.make_table_features_process(stock_set, interval, 8)

In [5]:
f.make_master_table(stock_set, interval)

Saved combined dataframe to: ./data_transformed/all_1d_model_df.pkl


In [ ]:
f.xg_boost_model(interval, grid_search_on=True)

/home/oem/Documents/github/stock_production_agg/stkenv/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [12:16:01] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/home/oem/Documents/github/stock_production_agg/stkenv/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [12:16:12] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/home/oem/Documents/github/stock_production_agg/stkenv/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [12:16:19] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/home/oem/Documents/github/stock_production_agg/stkenv/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [12:16:31] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.

In [7]:
f.model_prospect('nvda', '1d')


Model Prediction 1d NVDA:
Predicted Next 1d Movement: UP

Model Prediction Probabilities:
no_change (0): 0.3145
up (1): 0.3536
down (2): 0.3319

Last Entry 1d NVDA Datetime Used for Prediction:
NOTE: It's in or contains the full 1d time interval.
EST: 2025-01-15 12:00:00 AM EST-0500
PDT: 2025-01-14 09:00:00 PM PST-0800


In [ ]:
df_val_nvda = f.model_validation('nvda', '5m')
df_val_nvda.head(5)

In [ ]:
df_val_nvda.dir_pred_match.sum()

In [ ]:
df_val_nvda.shape

In [ ]:
20/50